The following code is for running QFASA_Dist in R

If you are new to R, instructions on how to download and install R can be found at <http://www.r-project.org>

Manuals with information on the R language and basic instructions for using R can be found at <https://cran.r-project.org/manuals.html> The manuals are available in HTML or as downloadable PDFs.  

The code and associated instructions for running QFASA_Dist are formatted within an RStudio markdown document.  
For instructions on how to download and install RStudio go to <https://www.rstudio.com>  
If you are new to RStudio, information on working with the interface can be found under 'Resources'

***

### I. the p.QFASA functions
QFASA_Dist code written by: Connie Stewart

* the optimization function was changed from the optim function used in the original QFASA.pack package (the R package Wade Blanchard developed) to the solnp function from the Rsolnp package
    + optimization functions were tested side by side and no difference was found in the behaviour of the functions - the results are virtually identical  
* code was added for:
    + the Aitchison distance (AIT) and the Chi-Squared Distance (CSD)
    + the value of the minimized distance and the contribution of each FA in the modelling set to the minimized distance
    
#### Required package: Rsolnp

In [3]:
library(Rsolnp)
#setwd("~/wa//github/QFASA/dev/")

### Load Required Functions (there are 3)  
#### Function: **MEANmeth**
Returns the multivariate mean FA signature of each prey group entered into the model. Result can be passed to prey.mat in p.QFASA   
Input: prey.mat --> matrix containing the FA signatures of the prey. The first column indexes the prey group.

In [4]:
MEANmeth <- function(prey.mat) {
  prey.means <- apply(prey.mat[, -1], 2, tapply, prey.mat[, 1], mean)
  return(prey.means)
}

#### Function: **mean.geometric**
Returns the geometric mean.  
Input: x--> vector

In [5]:
mean.geometric <- function(x) {
  D <- length(x)
  return(prod(x)^(1./D))
}

#### Function: **p.QFASA**
Computes the diet estimate for each predator in seal.mat using either the Kullback-Leibler Distance (KL), the Aitchison Distance (AIT) or the Chi-Squared Distance (CS) 

Inputs:  
seal.mat --> matrix containing the FA signatures of the predator  

prey.mat --> matrix containing a representative FA signature from each prey group (usually the mean) -> assumes that the first column contains the name of the prey group  

cal.mat --> matrix of calibration factors where the *i* th column is to be used with the *i* th seal  

dist.measure --> distance measure to use for estimation: 1=KL, 2=AIT or 3=CS  

gamma --> parameter required for calculations using CS distance (passed to CS.obj). Currently being set to =1.  


In [21]:
p.QFASA <- function(seal.mat, prey.mat, cal.mat, dist.meas, gamma, FC = rep(1., nrow(prey.mat)),
                    start.val = rep(0.99999, nrow(prey.mat)), ext.fa = ext.common.fa.list) {
  
   # CALIBRATING SEAL FA SIGNATURES AND THEN EXTRACTING EXTENDED DIETARY FAS
  if ((is.vector(cal.mat)) || (nrow(cal.mat) == 1.) || (ncol(cal.mat ) == 1.)) {
  
    # IF ONLY ONE SEAL
    
    seal.mat <- t(t(seal.mat)/as.vector(unlist(cal.mat)))
    seal.mat <- as.data.frame(seal.mat)[ext.fa]
    seal.mat <- seal.mat/apply(seal.mat, 1., sum)
    seal.mat <- as.matrix(seal.mat)
    
  } else {
    
    seal.mat <- seal.mat/t(cal.mat)
    seal.mat <- as.data.frame(seal.mat)[ext.fa]
    seal.mat <- seal.mat/apply(seal.mat, 1., sum)
    seal.mat <- as.matrix(seal.mat)
    
  }
    
  I <- nrow(prey.mat)
  ns <- nrow(seal.mat)
  p.mat <- matrix(rep(0, nrow(prey.mat) * nrow(seal.mat)), byrow = T, nrow(seal.mat),nrow(prey.mat))
  
  more.list <- vector("list",ns)
    
  # USING SAME STARTING VECTOR FOR EACH OF THE ns SEALS
  
  if (!(is.matrix(start.val))) {
    start.val <- matrix(rep(start.val, ns), byrow = T, ns, I)
  }
  
  if (dist.meas == 1) { # KL Distance
    
    for (i in 1.:nrow(seal.mat)) {
      p.all <- solnp(pars = start.val[i,  ], fun  = KL.obj, 
                     seal = seal.mat[i,  ], 
                     prey.quantiles = prey.mat, eqfun=QFASA.const.eqn, eqB=1,
                     LB = rep(0, nrow(prey.mat)),
                     UB = rep(0.999999, nrow(prey.mat)),control=list(trace=0))
      if (p.all$convergence!=0){
        
        cat("WARNING: DID NOT CONVERGE")
      }
          
      p.mat[i,  ] <- p.all$par
      
      more.list[[i]] <- KL.more(p.mat[i,],seal.mat[i,],prey.mat)
          
    }
    
  }
  else if (dist.meas==2) { # AIT distance
    for (i in 1.:nrow(seal.mat)) {
      p.all <- solnp(pars = start.val[i,  ], fun  = AIT.obj,
                     seal = seal.mat[i,  ],
                     prey.quantiles = prey.mat, eqfun=QFASA.const.eqn, eqB=1,
                     LB = rep(0, nrow(prey.mat)),
                     UB = rep(0.999999, nrow(prey.mat)),control=list(trace=0))
      if (p.all$convergence!=0){ 
        
        cat("WARNING: DID NOT CONVERGE")
      }
            
      p.mat[i,  ] <- p.all$par
      
      more.list[[i]] <- AIT.more(p.mat[i,],seal.mat[i,],prey.mat)
            
    } 
    
  }
  else { # CS distance
    for (i in 1.:nrow(seal.mat)) {
      
      p.all <- solnp(pars = start.val[i,  ], fun  = CS.obj,
                     seal = seal.mat[i,  ],
                     prey.quantiles = prey.mat, gamma = gamma, eqfun=QFASA.const.eqn, eqB=1,
                     LB = rep(0, nrow(prey.mat)),
                     UB = rep(0.999999, nrow(prey.mat)),control=list(trace=0))
      if (p.all$convergence!=0){
        
        cat("WARNING: DID NOT CONVERGE")
      }
      
      p.mat[i,  ] <- p.all$par
      
      more.list[[i]] <- CS.more(p.mat[i,],seal.mat[i,],prey.mat,gamma)
      
    }
    
  }
  
  if (is.matrix(FC))
  { FC.mat <- FC
  } else { 
    FC.mat <- matrix(rep(FC, nrow(seal.mat)), byrow = T, nrow(seal.mat), I)
    
  } 
  
  p.mat <- p.mat/FC.mat
  p.mat <- p.mat/apply(p.mat, 1, sum)
  
  out.list <- list(p.mat,more.list)
  names(out.list) <- c("Diet Estimates","Additional Measures")
  return(out.list)
}

KL.obj <- function(alpha, seal, prey.quantiles) {
  
  # AUGUST 12TH, 2014
  # SIMILAR TO optquantile.obj2 BUT DOES NOT NORMALIZE ALPHA.
  # THE OBJECTIVE FUNCTION TO BE MINIMIZED
  # INPUT:
  # alpha              --> VECTOR OVER WHICH MINIMIZATION TAKES PLACE.
  # seal                --> VECTOR OF  FATTY ACID COMPOSITIONS OF SEAL.
  # prey.quantiles  --> MATRIX OF FATTY ACID COMPOSITION OF PREY.
  #                               EACH ROW CONTAINS AN INDIVIDUAL PREY.
  #                               FROM A DIFFERENT SPECIES.
  
  no.zero <- sum(seal == 0.)
  seal[seal == 0.] <- 1e-05
  seal[seal > 0.] <- (1. - no.zero * 1e-05) * seal[seal > 0.]
  sealhat <- t(as.matrix(alpha)) %*% prey.quantiles
  no.zero <- sum(sealhat == 0.)
  sealhat[sealhat == 0.] <- 1e-05
  sealhat[sealhat > 0.] <- (1. - no.zero * 1e-05) * sealhat[sealhat >0.]
  return(KL.dist(seal, sealhat))
}

KL.dist <- function(x, y) {
  # SIMILAR TO KL BUT REQUIRES TWO VECTORS AS INPUT
  # RETURNS KULBACK-LIEBLER DISTANCE BETWEEN x AND y WHERE 
  # INPUT:
  # x --> VECTOR 
  # y --> vector
  return(sum((x - y) * log(x/y)))
}

KL.more <- function(alpha, seal, prey.quantiles){
  
  # OCTOBER 28TH, 2014
  # USED TO PROVIDE ADDITIONAL INFORMATION ON MODEL COMPONENTS WHEN
  # alpha CORRESPONDS TO THE QFASA DIET ESTIMATES (i.e. ESTIMATES
  # THAT MINIMIZED THE AIT DISTANCE.)
   
  # THE OBJECTIVE FUNCTION TO BE MINIMIZED
  
  # INPUT:
  # alpha              --> VECTOR OVER WHICH MINIMIZATION TAKES PLACE.
  # seal                --> VECTOR OF  FATTY ACID COMPOSITIONS OF SEAL.
  # prey.quantiles  --> MATRIX OF FATTY ACID COMPOSITION OF PREY.
  #                               EACH ROW CONTAINS AN INDIVIDUAL PREY.
  #                               FROM A DIFFERENT SPECIES.
  
  no.zero <- sum(seal == 0.)
  seal[seal == 0.] <- 1e-05
  seal[seal > 0.] <- (1. - no.zero * 1e-05) * seal[seal > 0.]
  
  sealhat <- t(as.matrix(alpha)) %*% prey.quantiles
  no.zero <- sum(sealhat == 0.)
  sealhat[sealhat == 0.] <- 1e-05
  sealhat[sealhat > 0.] <- (1. - no.zero * 1e-05) * sealhat[sealhat >0.]
  
  KL.vec <- (seal - sealhat) * log(seal/sealhat)
  
  dist <- sum(KL.vec)
  
  out.list <- list(sealhat,KL.vec,KL.vec/sum(KL.vec),dist)
  names(out.list) <- c("ModFAS","DistCont", "PropDistCont","MinDist")
  
  return(out.list)
}

AIT.obj <- function(alpha, seal, prey.quantiles) {
  
  # AUGUST 12TH, 2014
  # SIMILAR TO optcompdiff.obj BUT DOES NOT NORMALIZE ALPHA.
  # USED IN solnp AS THE OBJECTIVE FUNCTION TO BE MINIMIZED
  # INPUT:
  # alpha --> VECTOR OVER WHICH MINIMIZATION TAKES PLACE
  # seal --> VECTOR OF  FATTY ACID COMPOSITIONS OF SEAL
  # prey.quantiles  --> MATRIX OF FATTY ACID COMPOSITION OF PREY.
  #                               EACH ROW CONTAINS AN INDIVIDUAL PREY
  #                               FROM A DIFFERENT SPECIES.
  
  no.zero <- sum(seal == 0.)
  seal[seal == 0.] <- 1e-05
  seal[seal > 0.] <- (1. - no.zero * 1e-05) * seal[seal > 0.]
  
  sealhat <- t(as.matrix(alpha)) %*% prey.quantiles
  no.zero <- sum(sealhat == 0.)
  sealhat[sealhat == 0.] <- 1e-05
  sealhat[sealhat > 0.] <- (1. - no.zero * 1e-05) * sealhat[sealhat > 0.]
  
  return(AIT.dist(seal, sealhat))
}

AIT.dist <- function(x, bigX) {
  
  # NOTE THAT THIS FUNCTION IS DIFFERENT THAT compdiff BECAUSE IT TAKES SQUARE ROOT
  # compdiff IS USED IN CALCULATING DIET ESTIMATES.
  
  # COMPUTES THE DIFFERENCE BETWEEN TO VECTORS OF COMPOSITIONAL DATA
  # AS DESCRIBED IN AITCHISON (1992) "MEASURES OF COMPOSITIONAL DIFFERENCE"
  
  return(sqrt(sum((log(x/mean.geometric(x)) - log(bigX/mean.geometric(bigX)))^
                    2.)))
}

AIT.more <- function(alpha, seal, prey.quantiles) {
  
  # OCTOBER 28TH, 2014
  # USED TO PROVIDE ADDITIONAL INFORMATION ON MODEL COMPONENTS WHEN
  # alpha CORRESPONDS TO THE QFASA DIET ESTIMATES (i.e. ESTIMATES
  # THAT MINIMIZED THE AIT DISTANCE.)
  
  # THE OBJECTIVE FUNCTION TO BE MINIMIZED
  # INPUT:
  # alpha                --> VECTOR OVER WHICH MINIMIZATION TAKES PLACE
  # seal                  --> VECTOR OF  FATTY ACID COMPOSITIONS OF SEAL
  # prey.quantiles  --> MATRIX OF FATTY ACID COMPOSITION OF PREY.
  #                               EACH ROW CONTAINS AN INDIVIDUAL PREY
  #                               FROM A DIFFERENT SPECIES.
  
  no.zero <- sum(seal == 0.)
  seal[seal == 0.] <- 1e-05
  seal[seal > 0.] <- (1. - no.zero * 1e-05) * seal[seal > 0.]
  
  sealhat <- t(as.matrix(alpha)) %*% prey.quantiles
  no.zero <- sum(sealhat == 0.)
  sealhat[sealhat == 0.] <- 1e-05
  sealhat[sealhat > 0.] <- (1. - no.zero * 1e-05) * sealhat[sealhat > 0.]
  
  AIT.sq.vec <- 
    ( log(seal/mean.geometric(seal)) - log(sealhat/mean.geometric(sealhat)) )^2
  
  dist <- (sum(AIT.sq.vec))^(1/2)
  
  out.list <- list(sealhat,AIT.sq.vec,AIT.sq.vec/sum(AIT.sq.vec),dist)
  names(out.list) <- c("ModFAS","DistCont", "PropDistCont","MinDist")
  
  return(out.list)
}

CS.obj <- function(alpha, seal, prey.quantiles, gamma){
  
  # OCTOBER 16TH, 2014
  # SIMILAR TO AIT.obj and KL.obj BUT DOES NOT REQUIRE CHANGING ZEROS
  
  # THE OBJECTIVE FUNCTION TO BE MINIMIZED
  # INPUT:
  # alpha                --> VECTOR OVER WHICH MINIMIZATION TAKES PLACE.
  # seal                  --> VECTOR OF  FATTY ACID COMPOSITIONS OF SEAL.
  # prey.quantiles    --> MATRIX OF FATTY ACID COMPOSITION OF PREY.
  #                               EACH ROW CONTAINS AN INDIVIDUAL PREY.
  #                               FROM A DIFFERENT SPECIES.
  # gamma			--> PARAMETER PASSED TO chisq.dist
  
  sealhat <- t(as.matrix(alpha)) %*% prey.quantiles
  return(chisq.dist(seal,sealhat,gamma))
}

chisq.dist <- function(x.1,x.2,alpha) {
  
  # CALCULATES CS DISTANCE AS DESCRIBED IN TESTING FOR DIFF IN DIET PAPER.
  # THIS IS DIFFERENT THAN chisq.CA in DIRECTORY RUN.OCT11.R.1 BECAUSE
  # x1 AND x2 ARE NOT ASSUMED TO BE TRANSFORMED.
  
  nfa <- length(x.1)
  
  y.1 <- x.1^(alpha)
  y.1 <- y.1/sum(y.1)
  
  y.2 <- x.2^(alpha)
  y.2 <- y.2/sum(y.2)
  
  d.sq <- (y.1-y.2)^2
  c.vec <- y.1+y.2
  
  if ( any(d.sq!=0) ) {
    
    d.sq[d.sq!=0] <- d.sq[d.sq!=0]/c.vec[d.sq!=0]
  }
  
  CS.dist <- 1/alpha*sqrt(2*nfa)*sqrt(sum(d.sq))
  
  return(CS.dist)
  
}

CS.more <- function(alpha, seal, prey.quantiles, gamma){
  
  # OCTOBER 28TH, 2014
  # USED TO PROVIDE ADDITIONAL INFORMATION ON MODEL COMPONENTS WHEN
  # alpha CORRESPONDS TO THE QFASA DIET ESTIMATES (i.e. ESTIMATES
  # THAT MINIMIZED THE CS DISTANCE.)
  
  # THE OBJECTIVE FUNCTION TO BE MINIMIZED
  # INPUT:
  # alpha --> VECTOR OVER WHICH MINIMIZATION TAKES PLACE.
  # seal --> VECTOR OF  FATTY ACID COMPOSITIONS OF SEAL.
  # prey.quantiles --> MATRIX OF FATTY ACID COMPOSITION OF PREY.
  #                               EACH ROW CONTAINS AN INDIVIDUAL PREY.
  #                               FROM A DIFFERENT SPECIES.
  # gamma --> PARAMETER PASSED TO chisq.dist
  
  sealhat <- t(as.matrix(alpha)) %*% prey.quantiles
  
  nfa <- length(seal)
  
  y.1 <- seal^(gamma)
  y.1 <- y.1/sum(y.1)
  
  y.2 <- sealhat^(gamma)
  y.2 <- y.2/sum(y.2)
  
  
  d.sq <- (y.1-y.2)^2
  c.vec <- y.1+y.2
  
  if ( any(d.sq!=0) ) {
    
    d.sq[d.sq!=0] <- d.sq[d.sq!=0]/c.vec[d.sq!=0]
  }
  
  CS.vec.sq <- d.sq
  
  dist <- 1/gamma*sqrt(2*nfa)*sqrt(sum(d.sq))
  
  out.list <- list(sealhat,CS.vec.sq,CS.vec.sq/sum(CS.vec.sq),dist)
  names(out.list) <- c("ModFAS","DistCont", "PropDistCont","MinDist")
  
  return(out.list)
}

QFASA.const.eqn <- function(alpha,seal=seal.mat[i,],prey.quantiles=prey.mat, gamma=gamma) {
  
  return(sum(alpha))
  
  mean.geometric <- function(x) {
    
    # RETURNS GEOMETRIC MEAN
    
    # INPUT:
    #  x --> VECTOR
    
    D <- length(x)
    return(prod(x)^(1./D))
  }
  
}

multiplot <- function(..., plotlist=NULL, file, cols=1, layout=NULL) {
  require(grid)
  
  # Make a list from the ... arguments and plotlist
  plots <- c(list(...), plotlist)
  
  numPlots = length(plots)
  
  # If layout is NULL, then use 'cols' to determine layout
  if (is.null(layout)) {
    # Make the panel
    # ncol: Number of columns of plots
    # nrow: Number of rows needed, calculated from # of cols
    layout <- matrix(seq(1, cols * ceiling(numPlots/cols)),
                     ncol = cols, nrow = ceiling(numPlots/cols))
  }
  
  if (numPlots==1) {
    print(plots[[1]])
    
  } else {
    # Set up the page
    grid.newpage()
    pushViewport(viewport(layout = grid.layout(nrow(layout), ncol(layout))))
    
    # Make each plot, in the correct location
    for (i in 1:numPlots) {
      # Get the i,j matrix positions of the regions that contain this subplot
      matchidx <- as.data.frame(which(layout == i, arr.ind = TRUE))
      
      print(plots[[i]], vp = viewport(layout.pos.row = matchidx$row,
                                      layout.pos.col = matchidx$col))
    }
  }
}

### II. Modelling Inputs
#### Load Required Packages
* plyr package: the function 'ldply' from this package is used to compile the list output generated by the KL.more, AIT.more or CS.more functions (which give the various values for the contributions of individual FA to the minimized distance) into a single matrix
    * another package which is useful for summarizing the ouput but is not required is the 'doBy' package which has functions for very easily summarizing data by groups

In [22]:
library(plyr)

#### Create the Required Model Inputs (there are 6)  
* prior to starting make sure that:
    + the fatty acid names in all files are the same (contain the exact same numbers/characters and punctuation)
    + that there are no fatty acids in the prey file that do not appear in the predator file and visa versa

#### 1. Distance Measure
Choose from one of three distance measures:  
1=KL (Kullback-Leibler) || 2=AIT (Aitchison) || 3=CSD (Chi-Squared) 

In [23]:
dist.meas=1

#### 2. Fatty Acid Set  
* this is the list of FAs to be used in the modelling
* the simplest alternative is to load a .csv file which contains a single column with a header row and the names of the fatty acids listed below (see example file __"FAset.csv"__)
* a more complicated alternative is to load a .csv file with the full set of FAs and then add code to subset the FAs you wish to use from that set --> this alternative is useful if you are planning to test multiple FA sets
* regardless of how you load the FA set it must be converted to a vector 

In [24]:
fa.set=read.csv(file=system.file("exdata", "FAset.csv", package="QFASA"), as.is=TRUE)
fa.set=as.vector(unlist(fa.set))

#### 3. Matrix of Predator FA signatures  
* the FA signatures in the originating .csv file should be in percentages not proportions (i.e. adding to ~100 not 1)  
* each predator signature is a row with the FAs in columns (see example file __"predatorFAs.csv"__)
* the FA signatures are subsetted for the chosen FA set (created above) and renormalized during the modelling so there is no need to subset and/or renormalize prior to loading the .csv file or running p.QFASA  
    + BUT make sure that the the same FAs appear in the predator and prey files (if a FA appears in one but not the other the code will give you an error)
* unlike the original QFASApack code the predator FA .csv file can contain as much tombstone data in columns as you wish but the predator FA signatures must be extracted as a separate input in order to run in p.QFASA
    + For example: in the code below the predator .csv file ("predatorFAs.csv") has 4 tombstone columns (SampleCode, AnimalCode, SampleGroup, Biopsy). Prior to running QFASA the tombstone (columns 1-4) and FA data (columns 5 onward) are each extracted from the original data frame. The FA data becaomes the the predator.matrix (which is passed to p.QFASA) and the tombstone data is retained so that it can be recombined with the model output later on

In [33]:
predators=read.csv(file=system.file("exdata", "predatorFAs.csv", package="QFASA"), as.is=TRUE)
tombstone.info=predators[,1:4]
predator.matrix=predators[,5:(ncol(predators))]
npredator.matrix = nrow(predator.matrix)
# number of predator FA signatures this is used to create the matrix of CC values (see section 6 below)

#### 4. Matrix of Prey FA signatures  
* the FA signatures in the originating .csv file should be in percentages not proportions (i.e. adding to ~100 not 1)
* the prey file should contain all of the individual FA signatures of the prey and their lipid contents (where appropriate) - a matrix of the mean values for the FAs (prey.matrix) by the designated prey modelling group is then calculated using the MEANmeth function loaded above
* like the predator .csv file you can have as many tombstone data columns as required but there must be at least one column that identifies the modelling group to be used (in the example file used below __"preyFAs.csv"__ it is the "Species" column)
* unlike the predator data, the prey data is not subsetted and renomalized during the modelling so the prey file needs to be subsetted for the desired FA set (created above) and renormalized to sum to 1 prior to calculating the mean values (see code below)
    + example: in the code below the "preyFAs.csv" file has 3 tombstone columns. The full FA set is extracted from the data frame (columns 4 onward), subsetted for the FA set in use and then renormalized over 1. The modelling group names (the "Species" column in this case) is then added back to the subsetted and renormalized data (as the first column) and the average values calculated using the MEANmeth function. Note that for the MEANmeth function to work the modelling group name must be in the first column

In [43]:
prey=read.csv(file=system.file("exdata", "preyFAs.csv", package="QFASA"), as.is=TRUE) #full file
prey.sub=(prey[,4:(ncol(prey))])[fa.set] #extract prey FA only from data frame and subset them for the FA set designated above
prey.sub=prey.sub/apply(prey.sub,1,sum) #renormalize over 1
group=as.vector(prey$Species) #extract the modelling group names from the full file
prey.matrix=cbind(group,prey.sub) #add modelling group names to the subsetted and renormalized FAs
prey.matrix=MEANmeth(prey.matrix) #create an average value for the FA signature for each designated modelling group 

#### 5. Prey lipid content
* mean lipid content by modelling group is calculated from the full prey file using the modelling group as a summary variable (see code below)
* **Note:** if no lipid content correction is going to be applied then a vector of '1's of length equal to the number of modelling groups is used as the vector instead
    + i.e. FC=rep(1,nrow(prey.matrix))

In [31]:
FC=prey[,c(2,3)] #numbers are the column which identifies the modelling group, and the column which contains the lipid contents
FC=as.vector(tapply(FC$lipid,FC$Species,mean,na.rm=TRUE))

#### 6. Calibration Coefficients    
* originating .csv file should contain 2 columns (with headers). The first contains the FA names, the second the value of the CC for each FA (see example file __"CC.csv"__)
* __IMPORTANT:__ the FAs in the CC.csv file __MUST__ be exactly the same as the FAs in the originating predator.csv file __AND__ they __MUST__ BE IN THE __*EXACT*__ SAME ORDER

In [41]:
cal=read.csv(file=system.file("exdata", "CC.csv", package="QFASA"), as.is=TRUE)
cal.vec=cal[,2]
cal.mat=replicate(npredator.matrix, cal.vec)

### III. Run p.QFASA

In [38]:
QFASA = p.QFASA(predator.matrix, 
                prey.matrix, 
                cal.mat, 
                dist.meas, 
                gamma=1, 
                FC, 
                start.val=rep(1,nrow(prey.matrix)), 
                fa.set)

### IV. p.QFASA Output
* the QFASA output is a list with 2 components:
    + Diet Estimates
    + Additional Measures

#### 1.Diet Estimates
* this is a matrix of the diet estimate for each predator (by rows, in the same order as the input file) by the modelling groups (by column, in the same order as the prey.matrix file)
* the estimates are expressed as a proportion (they will sum to 1)
* in the code below the Diet Estimate matrix is extracted from the QFASA output and the modelling group identities and predator tombstone data (created above) are added to the matrix 

In [39]:
DietEst = QFASA$'Diet Estimates'
DietEst = round(DietEst*100,digits=2) #estimates changed from proportions to percentages
colnames(DietEst) = (as.vector(rownames(prey.matrix)))
DietEst = cbind(tombstone.info, DietEst)
DietEst
#write.csv(DietEst, file="DietEst.csv", row.names=FALSE)

,SampleCode,AnimalCode,SampleGroup,Biopsy,capelin,coho,eulachon,herring,mackerel,pilchard,pollock,sandlance,squid,surfsmelt_lg,surfsmelt_s
1,3-01A,P031,T,A,35.3,0,0,44.9,9.25,2.36,0,8.19,0,0,0
2,3-01B,P031,T1,B,43.06,0,0,44.79,3.72,4.25,0,4.18,0,0,0
3,3-01C,P031,T1,C,50.15,0,0,34.34,6.14,3.97,0,5.4,0,0,0
4,3-02A,P032,T,A,37.8,0,0,47.14,1.42,5.04,0,8.6,0,0,0
5,3-02B,P032,T1,B,39.86,0,0,45.5,3.69,5.77,0,5.18,0,0,0
6,3-02C,P032,T1,C,47.99,0,0,35.59,4.47,5.39,0,6.55,0,0,0
7,3-04A,P034,T,A,41.29,0,0,46.33,8.24,2.65,0,1.49,0,0,0
8,3-04B,P034,T2,B,31.96,0,0,28,2.95,2.4,0,34.7,0,0,0
9,3-04C,P034,T2,C,21.07,0,0,14.45,0.27,2,0,37.03,0,25.17,0
10,3-05A,P035,T,A,28.91,0,0,63.5,0,3.93,0,3.66,0,0,0


#### 2.Additional Measures
* this is a list of lists where each list (one per predator) is itself a list of four outputs:
    + 1. ModFAS - the value of the modelled FA (i.e. after CCs have been applied and the FA subsetted and renormalised over the designated FA set). These are expressed as proportions (they will sum to 1).
    + 2. DistCont - the contribution of each FA to the final minimized distance
    + 3. PropDistCont - the contribution of each FA to the final minimized distance as a proportion of the total
    + 4. MinDist - the final minimized distance
* in the code below the 'ldply' function from the plyr package is used to compile the lists within 'Additional Measures' into a data frame with one row per predator (in the same order as the input predator matrix) and the values for each of the 4 lists arranged into columns. The 'ldply' function automatically names the columns of the data frame with a concatenation of the originating list name and the FA name so that the 4 sets of outputs can be easily identified within the data frame.

In [56]:
Add.meas = plyr::ldply(QFASA$'Additional Measures', data.frame)
Add.meas
#write.csv(Add.meas, file="AdditionalMeasures.csv", row.names=FALSE)

,ModFAS.c14.0,ModFAS.c16.0,ModFAS.c16.1w7,ModFAS.c16.2w6,ModFAS.c16.2w4,ModFAS.c16.3w6,ModFAS.c17.0,ModFAS.c16.3w4,ModFAS.c16.4w3,ModFAS.c16.4w1,ellip.h,PropDistCont.c22.1w11,PropDistCont.c22.1w9,PropDistCont.c22.1w7,PropDistCont.c21.5w3,PropDistCont.c22.4w6,PropDistCont.c22.5w6,PropDistCont.c22.4w3,PropDistCont.c22.5w3,PropDistCont.c22.6w3,MinDist
1,0.05476644,0.1853966,0.07018869,0.0009476068,0.001785397,0.006904514,0.005668164,0.005223226,0.00136927,0.009327573,⋯,0.1901595,5.655054e-07,0.03211225,0.001967619,0.0001127606,0.001347796,0.00103826,0.05188123,0.1323003,0.1117455
2,0.05504601,0.1841167,0.0720054,0.000993018,0.001630403,0.007083002,0.005335998,0.005632179,0.001257164,0.01046156,⋯,0.2144305,0.0002948707,0.02886522,0.00120424,0.000486005,0.001393266,0.0009902027,0.0568754,0.1062608,0.1138759
3,0.0560279,0.178989,0.07237388,0.0009591416,0.001744374,0.006727256,0.004916324,0.005371951,0.00127024,0.01013042,⋯,0.280748,0.0002576719,0.04609893,0.0003498474,0.0001097908,0.0007675461,0.0004645217,0.05127924,0.07952491,0.101763
4,0.05493239,0.1886819,0.07139458,0.001022593,0.001705438,0.007174932,0.005350123,0.005816044,0.001265329,0.01094744,⋯,0.1181795,0.0006784104,0.02146616,0.002255509,1.032331e-06,0.001122699,0.0008888684,0.06858449,0.1114991,0.1179728
5,0.05515642,0.1860906,0.07185312,0.001030936,0.001632665,0.00721221,0.005282616,0.005908598,0.001242121,0.0113276,⋯,0.1876884,0.0004740845,0.02271602,0.002011952,0.0001654532,0.001439437,0.0004898145,0.07777217,0.1045025,0.09947479
6,0.05605055,0.1817995,0.07238149,0.001002894,0.001735334,0.00689255,0.004853736,0.005695377,0.001242551,0.01105858,⋯,0.238434,0.0008247062,0.04597609,0.0002588143,0.0006368737,0.0008177795,0.0001430567,0.08096786,0.08163621,0.07960839
7,0.05470009,0.1817555,0.0715331,0.0009510172,0.001596014,0.007016661,0.005637892,0.005322179,0.001304027,0.009508294,⋯,0.2231776,0.0002455222,0.0352442,0.001378618,0.0003587328,0.00119998,0.0008454274,0.07180423,0.1259598,0.1304458
8,0.05638245,0.1968077,0.0671719,0.0009390731,0.002637647,0.006136985,0.00479621,0.004893719,0.001523644,0.009062321,⋯,0.1264499,0.007954542,0.05131047,0.002811708,6.604011e-05,0.0006568539,0.002530186,0.06909156,0.08884138,0.07461399
9,0.05600894,0.1993966,0.06217635,0.0009045445,0.003659729,0.005494125,0.004348029,0.004496314,0.00175547,0.008407243,⋯,0.09314936,0.004612156,0.07038415,0.001801226,0.0001923915,0.002861917,0.002387933,0.0519379,0.03896595,0.05032101
10,0.05360442,0.1916365,0.07110096,0.001024003,0.001520968,0.007506484,0.005982933,0.00585472,0.001266523,0.01050922,⋯,0.08238331,1.986549e-08,0.03308884,0.001871621,1.585309e-05,0.000439642,0.0008685825,0.06503273,0.100723,0.09207496
